In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns

In [6]:
import  torch.utils.tensorboard as tensorboard
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('runs/Final_Inference')

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
device

device(type='cpu')

In [5]:
data_dir = 'PetImages'
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [6]:
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=16, shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

In [7]:
samples=iter(dataloaders['train'])
img,label=next(samples)

img=img.to(device)
label=label.to(device)

image_grid=torchvision.utils.make_grid(img,nrow=8)
writer.add_image('images',image_grid)
writer.close()


print(f"Image shape: {img.shape} -> [batch_size, color_channels, height, width]")
print(f"Label shape: {label.shape}")

Image shape: torch.Size([16, 3, 224, 224]) -> [batch_size, color_channels, height, width]
Label shape: torch.Size([16])


In [8]:
from tqdm import tqdm
def train(model,criterion,optimizer,dataloaders,dataset_sizes,device):
    model.train()
    running_loss = 0.0

    for inputs, labels in tqdm(dataloaders['train']):
        inputs,labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    epoch_loss = running_loss / dataset_sizes['train']
    print(f'Train Loss: {epoch_loss:.4f}')
    return epoch_loss

In [9]:
def validate(model,dataloaders,dataset_size,criterion,device):
    model.eval()

    running_loss = 0.0
    correct = 0
    total = 0
    all_labels = []
    all_preds = []
    tp=0
    tn=0
    fp=0
    fn=0
    precision = 0.0
    recall = 0.0
    f1 = 0.0


    with torch.no_grad():
        for inputs, labels in tqdm(dataloaders['val']):
            inputs,labels = inputs.to(device),labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()

            _,predicted = torch.max(outputs.data,1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

            for p, lbl in zip(predicted, labels):
                if p == lbl:
                    if p == 1:
                        tp += 1
                    else:
                        tn += 1
                else:
                    if p == 1:
                        fp += 1
                    else:
                        fn += 1

    epoch_loss = running_loss/dataset_size['val']
    accuracy = correct/total


    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * (precision * recall) / (precision + recall)

    conf_matrix = confusion_matrix(all_labels, all_preds)

    print(f'Validation Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')
    print(f'Confusion Matrix:\n{conf_matrix}')
    return epoch_loss, accuracy, precision, recall, f1, conf_matrix


In [10]:
import torch
import torchvision.models as models
from torchvision.models import ResNet18_Weights

weights_path = "C:/Users/awael/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth"

In [11]:
model_ft = models.resnet18()
model_ft.load_state_dict(torch.load(weights_path))
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.0001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [12]:
writer.add_graph(model_ft,img)
writer.close()

In [13]:
def plot_confusion_matrix():
  fig, ax = plt.subplots(figsize=(10, 10))
  sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', ax=ax)
  plt.xlabel('Predicted')
  plt.ylabel('True')
  plt.title('Confusion Matrix')

  return fig

In [14]:
num_epochs = 26
train_losses = []
val_losses = []
val_accuracies = []
val_precisions = []
val_recalls = []
val_f1s = []

best_f1 = 0.0
best_precision = 0.0
best_recall = 0.0
best_accuracy = 0.0

start_time = time.time()


for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    print('-' * 10)

    train_loss = train(model_ft, criterion, optimizer_ft, dataloaders, dataset_sizes, device)
    val_loss, val_accuracy, precision, recall, f1, conf_matrix = validate(model_ft, dataloaders, dataset_sizes, criterion, device)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)
    val_precisions.append(precision)
    val_recalls.append(recall)
    val_f1s.append(f1)

    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_f1 = f1
        best_precision = precision
        best_recall = recall

    if epoch%2==0:
        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('Loss/val', val_loss, epoch)
        writer.add_scalar('Accuracy/val', val_accuracy, epoch)
        writer.add_scalar('Precision/val', precision, epoch)
        writer.add_scalar('Recall/val', recall, epoch)
        writer.add_scalar('F1 Score/val', f1, epoch)
        writer.add_figure('Confusion Matrix', plot_confusion_matrix(), epoch)


end_time = time.time()
total_time = end_time - start_time
print(f'Training complete in {total_time // 60:.0f}m {total_time % 60:.0f}s')
print(f'Best Validation Accuracy: {best_accuracy:.4f}')
print(f'Best Precision: {best_precision:.4f}')
print(f'Best Recall: {best_recall:.4f}')
print(f'Best F1 Score: {best_f1:.4f}')

Epoch 1/26
----------


100%|██████████████████████████████████████████████████████████████████████████████| 1313/1313 [15:56<00:00,  1.37it/s]


Train Loss: 0.0140


100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [01:17<00:00,  3.24it/s]


Validation Loss: 0.0031, Accuracy: 0.9850
Precision: 0.9855, Recall: 0.9845, F1 Score: 0.9850
Confusion Matrix:
[[1971   29]
 [  31 1969]]
Epoch 2/26
----------


...

Validation Loss: 0.0013, Accuracy: 0.9925
Precision: 0.9940, Recall: 0.9910, F1 Score: 0.9925
Confusion Matrix:
[[1988   12]
 [  18 1982]]
Epoch 26/26
----------


100%|██████████████████████████████████████████████████████████████████████████████| 1313/1313 [16:01<00:00,  1.37it/s]


Train Loss: 0.0047


100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [01:15<00:00,  3.32it/s]

Validation Loss: 0.0013, Accuracy: 0.9920
Precision: 0.9900, Recall: 0.9940, F1 Score: 0.9920
Confusion Matrix:
[[1980   20]
 [  12 1988]]
Training complete in 449m 4s
Best Validation Accuracy: 0.9930
Best Precision: 0.9935
Best Recall: 0.9925
Best F1 Score: 0.9930


In [15]:
torch.save(model_ft.state_dict(), 'model_ft1.pth')

In [6]:
%load_ext tensorboard
%tensorboard --logdir="runs"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 10764), started 2 days, 1:04:48 ago. (Use '!kill 10764' to kill it.)

In [5]:
%reload_ext tensorboard